In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/train',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary',
    subset='training')
val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/train',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary',
    subset='validation')


model.fit(train_generator, validation_data=val_generator, epochs=10)

In [ ]:

model.save('/content/drive/MyDrive/face_recognition_model.h5')


def test_single_image(image_path, model):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    if len(faces) == 0:
        print("No faces detected in the image")
        return
    
    for (x, y, w, h) in faces:
        face_img = img[y:y+h, x:x+w]
        face_img_resized = cv2.resize(face_img, (64, 64))
        face_img_normalized = face_img_resized / 255.0
        face_img_input = np.expand_dims(face_img_normalized, axis=0)
        
        prediction = model.predict(face_img_input)[0][0]
        label = "Authorized" if prediction > 0.5 else "Unauthorized"
        confidence = prediction if prediction > 0.5 else (1 - prediction)
        
        color = (0, 255, 0) if prediction > 0.5 else (0, 0, 255)
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, f"{label} ({confidence:.2f})", (x, y-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        
        print(f"Prediction: {label}, Confidence: {confidence:.2f}")
    

    from google.colab.patches import cv2_imshow
    cv2_imshow(img)


test_single_image('/content/drive/MyDrive/dataset/train/Authorized/IMG_20250905_153355.jpg', model)
